In [1]:
import os
import re

In [8]:
def update_fit(path, prior_filename, pro_filename, check_slope=False, median_filename=None):
    '''
    Reads the fit directory to update lightcurve indices and check slope significance. Warning: this will overwrite the prior and procedures files!

    Parameters:
    -----------
    path : str
        Path to the directory containing the prior file and lightcurve files.
    prior_filename : str
        Name of the prior file to be updated.
    pro_filename : str
        Name of the procedures file to be updated.
    check_slope : bool, optional
        If True, checks the median file to see if the slope parameter is consistent with zero and removes it from the prior and procedures files if so. Default is False.
    median_filename : str, optional
        Name of the median file to be checked if check_slope is True.
    -----------
    Returns:
    None
    -----------

    '''

    lc_list = sorted([f for f in os.listdir(path) if f.startswith('n2') and f.endswith('.dat')])
    tess_indices = [i for i, f in enumerate(lc_list) if 'TESS' in f]

    with open(os.path.join(path, prior_filename), 'r') as f:
        priorfile_old = f.read()
        priorfile_new = priorfile_old

    # find and replace dilute indices
    for index in tess_indices:
        if index == 0:
            continue
        else:
            # update the indices for the starting values for dilute, variance, and f0, one-by-one
            # I include the format of the dilute line in the patterns for variance and f0 to ensure that only the old indices are replaced
            priorfile_new = re.sub(r'(variance_\d+)( -?\d+.\d+\ndilute_\d+ -?\d+)', f'variance_{index}' + r'\2', priorfile_new, count=1)
            priorfile_new = re.sub(r'(dilute_\d+ -?\d+.\d+\n)(f0_\d+ )', r'\1' + f'f0_{index} ', priorfile_new, count=1)
            priorfile_new = re.sub(r'dilute_\d+ -?\d+.\d+', f'dilute_{index} dilute_0 0', priorfile_new, count=1) # do dilute last b/c of above patterns
    if 0 in tess_indices:
        priorfile_new = re.sub(r'dilute ', 'dilute_0 ', priorfile_new) # update the first dilute parameter
        priorfile_new = re.sub(r'variance ', 'variance_0 ', priorfile_new) # update the first variance parameter
        priorfile_new = re.sub(r'f0 ', 'f0_0 ', priorfile_new) # update the first f0 parameter

    if check_slope:
        # read median file and check if slope is consistent with zero
        with open(f'{path}/fitresults/{median_filename}', 'r') as f:
            medianfile = f.read()
            slope_params = re.findall(r'slope_0,(-?\d*\.\d+),(\d*\.\d+),(\d*\.\d+)', medianfile) # format: parameter, median, upper_err, lower_err
            median_slope = float(slope_params[0][0])
            median_slope_upp_err = float(slope_params[0][1])
            median_slope_low_err = float(slope_params[0][2])

            if (median_slope - median_slope_low_err <= 0) and (median_slope + median_slope_upp_err >= 0):
                remove_slope = True
            else:
                remove_slope = False
            
        if remove_slope:
            # remove slope starting value from prior file
            priorfile_new = re.sub(r'slope -?\d+.\d+', '', priorfile_new)
            
    
    with open(os.path.join(path, prior_filename), 'w') as f:
        f.write(priorfile_new)

    ## Update procedures file

    # initialize lists
    exptimes = []
    ninterp = []
    fitdilute = []

    for i in range(len(lc_list)):
        if i in tess_indices:
            exptime_s = re.findall(r'(\d+)sunbinned.dat', lc_list[i])
            exptime_min = int(exptime_s[0]) / 60
            exptimes.append(int(exptime_min))
            if exptime_min == 30:
                ninterp.append(10)
            elif exptime_min == 10:
                ninterp.append(4)
            else:
                ninterp.append(1)
            fitdilute.append(1)
        else:
            exptimes.append(1)
            ninterp.append(1)
            fitdilute.append(0)

    with open(os.path.join(path, pro_filename), 'r') as f:
        pro_file_old = f.read()

    pro_file_new = re.sub(r'exptime=\[.*\]', f'exptime={exptimes}', pro_file_old)
    pro_file_new = re.sub(r'ninterp=\[.*\]', f'ninterp={ninterp}', pro_file_new)
    pro_file_new = re.sub(r'fitdilute=\[.*\]', f'fitdilute={fitdilute}', pro_file_new)

    if check_slope:
        if remove_slope:
            pro_file_new = re.sub(r'/fitslope', '', pro_file_new)
            print('Slope removed from fit as it is consistent with zero.')
        else:
            print(f'Slope retained in fit as it appears to be significant (median: {median_slope}, +{median_slope_upp_err}, -{median_slope_low_err}).')

    with open(os.path.join(path, pro_filename), 'w') as f:
        f.write(pro_file_new)

In [9]:
def update_slurm_file(path, slurm_filename, time_days, ncpu):
    with open(os.path.join(path, slurm_filename), 'r') as f:
        slurm_file_old = f.read()
    
    # Set time limit for SLURM job
    time_minutes = time_days * 24 * 60
    slurm_file_new = re.sub(r'#SBATCH --time=\d+', f'#SBATCH --time={int(time_minutes)}', slurm_file_old)

    # Set time limit for EXOFASTv2
    time_seconds = (time_minutes - 360) * 60 # EXOFAST ends 6 hrs before SLURM to allow for transit time calculations
    slurm_file_new = re.sub(r'maxtime=\d+', f'maxtime={int(time_seconds)}', slurm_file_new)

    if 'makepdfs.sh' not in slurm_file_new:
        # Append a few new lines at the bottom to turn postscripts into pdfs
        slurm_file_new += "\n\n# Turn postscripts into pdfs\n"
        slurm_file_new += "cd fitresults\n"
        slurm_file_new += "makepdfs.sh\n"
        slurm_file_new += "cd ..\n"

    # Set number of CPUs
    slurm_file_new = re.sub(r'#SBATCH --cpus-per-task=\d+', f'#SBATCH --cpus-per-task={ncpu}', slurm_file_new) # for SLURM
    slurm_file_new = re.sub(r'nthread=\d+', f'nthread={ncpu}', slurm_file_new) # for EXOFASTv2

    with open(os.path.join(path, slurm_filename), 'w') as f:
        f.write(slurm_file_new)

In [10]:
path = '/mnt/research/Exoplanet_Lab/jack/Global_Fits/meep3/toi5479'
prior_filename = '67444896.priors.2'
pro_filename = 'fittoi5479.pro'
median_filename = '67444896.median.csv'

update_fit(path, prior_filename, pro_filename, check_slope=True, median_filename=median_filename)
update_slurm_file(path, 'SLURM_toi5479.sb', time_days=4, ncpu=25)

Slope retained in fit as it appears to be significant (median: 0.068, +0.066, -0.066).
